## Setup


In [10]:
import os
import json
import requests

API_URL = 'https://dev.api.app.genei.io/graphql'
API_KEY = 'sk_12f9accc6ff84c23a9cd1a67818fab0b'
HEADERS = {
    'Content-Type': 'application/json',
    'x-api-key': API_KEY
}

data_dir_path = 'data/'
dist_dir_path = 'dist/'

## Retrieve Genei note data


### Search Genei notes


In [6]:
note_name = "1.1 - The Routledge Handbook of Ethics and Public Policy, Chapter 4, Weale, Albert, 'Public Policy and Normative Methods' - Chapter Summary"

# Search for notes in Genei by name
query = """
    query {{
        searchByName(input: {{
            name: "{}"
        }}) {{
            results {{
                item {{
                    __typename
                    ... on Note {{
                        id
                        name
                    }}
                }}
                score
            }}
        }}
    }}
""".format(note_name)
try:
    response = requests.post(API_URL, json={'query': query}, headers=HEADERS)
    # Raise an exception if the response status code is not 200
    response.raise_for_status()
    search_results = response.json()['data']['searchByName']['results']
except requests.exceptions.RequestException as e:
    print(f"An error occurred: {e}")
    search_results = []

top_search_result_id = None
if len(search_results) > 0:
    top_search_result_id = search_results[0]['item']['id']

top_search_result_id

'note_cl8ln255w000001md47a6h6cg'

### Return note data from top search result


In [12]:
if top_search_result_id:
    query = """
    query {{
        note(id: "{}") {{
            data
        }}
    }}
    """.format(top_search_result_id)
    response = requests.post(API_URL, json={'query': query}, headers=HEADERS)
    note = response.json()
    note_data = note['data']['note']['data']['root']

    if not os.path.exists(data_dir_path):
        os.makedirs(data_dir_path)

    file_name = os.path.join(data_dir_path, note_name + '.json')
    with open(file_name, 'w') as f:
        json.dump(note_data, f)

## Convert Genei note data to markdown


In [45]:
def json_to_md(node):
    if node['type'] == 'text' or node['type'] == 'code-highlight':
        return node['text']

    if node['type'] == 'heading':
        level = int(node['tag'][1])
        return f"{'#' * level} " + ' '.join(
            [json_to_md(child) for child in node['children']]) + "\n"

    if node['type'] == 'list':
        return '\n'.join([json_to_md(child) for child in node['children']])

    if node['type'] == 'listitem':
        prefix = "- "
        if node.get('checked'):
            prefix = "- [x] "
        return prefix + ' '.join([json_to_md(child) for child in node['children']])

    if node['type'] == 'code':
        return f"```{node.get('language', '')}\n" + ''.join(
            [json_to_md(child) for child in node['children']]) + "\n```"

    if node['type'] == 'paragraph' or node['type'] == 'root':
        return '\n'.join([json_to_md(child) for child in node['children']])

    return ''


markdown_output = json_to_md(note_data)
with open('{}.md'.format(note_name), 'w') as f:
    f.write(markdown_output)